In [1]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
from nltk import Tree
import re

In [2]:
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/elmo-constituency-parser-2020.02.10.tar.gz")


In [3]:
def replace_zone(index, node, tree):
    length = len(tree)
    if len(node.leaves()) < 2 and index+1 < length:
        next_node = tree[index+1]
        result = [node.leaves()[0], next_node.leaves()[0]]
        return result
    else:
        return node.leaves()[0:2]

In [4]:
def fix_sentence_verb_tag(sentence, tree):
    length = len(tree)
    fixed = False
    vp_exists = False
    for index in range(length):
        node = tree[index]
        if node.label() == 'VP':
            vp_exists = True
            exists = False
            for pair in node.pos():
                if pair[1] == 'VB' or pair[1] == 'VBP':
                    exists = True
                    break
            if not exists:
                for pair in node.pos():
                    if pair[1] != 'CC' and pair[1] != 'RB':
                        to_replace = ' '.join(map(str, replace_zone(index, node, tree)))
                        replaced = to_replace.replace(pair[0], pair[0]+' the')
                        sentence = sentence.replace(to_replace, replaced)
                        fixed = True
                        break   
    
    if not vp_exists:
        for pair in tree[0].pos():
            if pair[1] != 'CC' and pair[1] != 'RB':
                to_replace = ' '.join(map(str, replace_zone(0, tree[0], tree)))
                replaced = to_replace.replace(pair[0], pair[0]+' the')
                sentence = sentence.replace(to_replace, replaced)
                fixed = True
                break  
                
    return sentence, fixed

In [17]:
def save_to_file(pos, words):
    f = open(pos+".txt", "a")
    for word in words:
        f.write(word + '\n')
    f.close()

In [18]:
def save_words(tree):
    verbs = re.findall("VBP? (\w+)",tree)
    adverbs = re.findall("RB? (\w+)", tree)
    adjectives = re.findall("JJ? (\w+)", tree)
    
    save_to_file('verbs2', verbs)
    save_to_file('adverbs2', adverbs)
    save_to_file('adjectives2', adjectives)

In [19]:
def process_sentence(sentence):
    result = predictor.predict_json({"sentence": sentence})
    tree = Tree.fromstring(result['trees'])
    new_sentence, fixed = fix_sentence_verb_tag(sentence, tree)
    if fixed:
        result = predictor.predict_json({"sentence": new_sentence})
        tree = Tree.fromstring(result['trees'])
    save_words(result['trees'])

In [20]:
import pandas as pd

In [21]:
recipe_file_path = "../csvs/steps2.csv"
df = pd.read_csv(recipe_file_path, delimiter=',')

In [22]:
no_columns = len(df.columns)
if no_columns >= 4:
    df.drop(df.columns[[i for i in range(4,no_columns)]], axis=1, inplace=True)
df.head()

,id,content,number,recipe_id
0,1153,Preheat oven to 200°F. Sprinkle 1 side of tuna...,1,353
1,1154,"Add butter, sliced green onions, cilantro, gin...",2,353
2,1155,Preheat oven to 350°F. Butter and flour two 9x...,1,354
3,1156,Divide batter equally between prepared pans. B...,2,354
4,1157,Preheat oven to 300°F. and grease pans. Line b...,1,355


In [23]:
steps = df['content'].tolist()
steps = [step.lower() for step in steps if isinstance(step, str)]

In [24]:
len(steps)

49991

In [25]:
import time

In [30]:
i_start = 5000
i_end = 10000

In [31]:
start = time.time()

for index in range(i_start,i_end): 
    try:
        step = steps[index]
        step = step.replace('(', '')
        step = step.replace(')', '')
        sentences = re.split('(?<=[.!?;]) +',step)
        for sentence in sentences:
            process_sentence(sentence)
    except:
        print(index, step)
        traceback.print_exc()
        break
        
end = time.time()
print("total: ", str(end - start))
time_sentence = (end - start) / (i_end - i_start)
print("per step: ", str(time_sentence))

7416 boil port and wine in heavy small saucepan until mixture is reduced to 2/3 cup, about 8 minutes. set aside.


NameError: name 'traceback' is not defined